### **Scenario 1**

####A retail company receives daily updates for its product catalog, including new products, price changes, and discontinued items. Instead of overwriting the entire catalog or simply appending new records, they need to upsert the incoming data-updating existing products with the latest information and inserting new products-ensuring the catalog remains accurate and up-to-date in real-time.

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
%sql
select * from delta.sample.products;

### dedup the value based on updatedtime before upsert

In [0]:
df = spark.sql('select * from delta.sample.products')

In [0]:
df = df.withColumn('dedup',row_number().over(Window.partitionBy('id').orderBy(col('updatedDate').desc())))
df = df.filter(col('dedup')==1).drop("dedup")

In [0]:
if len(dbutils.fs.ls('/Volumes/main/volume/task/pyspark_products/'))>1:
    dlt_obj = DeltaTable.forPath(spark,'/Volumes/main/volume/task/pyspark_products/')

    dlt_obj.alias('trg').merge(
        df.alias('src'),
        'src.id = trg.id')\
        .whenMatchedUpdateAll(condition = "src.updatedDate > trg.updatedDate")\
        .whenNotMatchedInsertAll()\
        .execute()
else:
    df.write.format('delta').mode('overwrite').save('/Volumes/main/volume/task/pyspark_products/')


In [0]:
%sql
select * from delta.`/Volumes/main/volume/task/pyspark_products/`

### Scenario **2**

####An e-commerce platform receives customer order details from its mobile application in JSON format through a streaming pipeline. The JSON contains nested fields such as customer information, payment details, and a list of purchased items. To store and analyze this data efficiently in a data warehouse, the nested structure must be flattened into a tabular format using PySpark, ensuring all relevant attributes are readily accessible reporting and analytics.

In [0]:
df = spark.read.format('json').option('inferSchema',True).option('multiLine',True).load('/Volumes/main/volume/task/apiData/')
df.display()

In [0]:
df_cust_update = (
    df
    .withColumn('delivery_updates', explode('delivery_updates'))
    .withColumn('items', explode('items'))
    .select(
        'customer.customer_id',
        'customer.email',
        'customer.location.city',
        'customer.location.country',
        'delivery_updates',
        'items'
    )
)

df_cust_update.display()
